In [18]:
from translate import Translator
import numpy as np
import preprocessor as p
import re
import codecs
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.snowball import SpanishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from translate import Translator

# Load Original Training Data

In [3]:
with open("/Users/ziranmin/Desktop/Assignment5/train/english_train.text", 'r') as f:
    en_train_texts = [l.strip() for l in f]

In [4]:
with open('/Users/ziranmin/Desktop/Assignment5/train/english_train.labels', 'r') as f:
    en_train_labels = [int(l.strip()) for l in f]

In [5]:
with open("/Users/ziranmin/Desktop/Assignment5/train/spanish_train.text", 'r') as f:
    sp_train_texts = [l.strip() for l in f]

In [6]:
with open('/Users/ziranmin/Desktop/Assignment5/train/spanish_train.labels', 'r') as f:
    sp_train_labels = [int(l.strip()) for l in f]

# Load Original Testing Data

In [7]:
with open("/Users/ziranmin/Desktop/Assignment5/test/english_test.text", 'r') as f:
    en_test_texts = [l.strip() for l in f]

In [8]:
with open("/Users/ziranmin/Desktop/Assignment5/test/english_test.labels", 'r') as f:
    en_test_labels = [int(l.strip()) for l in f]

In [9]:
with open("/Users/ziranmin/Desktop/Assignment5/test/spanish_test.text", 'r') as f:
    sp_test_texts = [l.strip() for l in f]

In [10]:
with open("/Users/ziranmin/Desktop/Assignment5/test/spanish_test.labels", 'r') as f:
    sp_test_labels = [int(l.strip()) for l in f]

# Load Mapping Labels¶

In [11]:
with open("/Users/ziranmin/Desktop/Assignment5/mapping/english_mapping.txt", 'r') as f:
    en_mapping = [l.strip() for l in f]

In [12]:
with open("/Users/ziranmin/Desktop/Assignment5/mapping/spanish_mapping.txt", 'r') as f:
    sp_mapping = [l.strip() for l in f]

# Remove Labels that only appears in English or Spanish Tweets

In [13]:
en_remove_list = [4, 10, 12, 14, 15, 17, 18]

In [14]:
sp_remove_list = [6,  7,  8,  14,  16,  17]

In [15]:
def remove_unmatched_label(text, label, remove_list):
    all_index_to_remove = []
    for i in remove_list:
        train_has_one_of_unmatched_index =  [index for index, value in enumerate(label) if value == i]
        all_index_to_remove += train_has_one_of_unmatched_index
    all_index_to_remove.sort()
    
    texts_removed = [text[i] for i in range(len(text)) if i not  in all_index_to_remove]
    labels_removed = [label[i] for i in range(len(label)) if i not  in all_index_to_remove]
    
    return texts_removed, labels_removed

In [114]:
en_train_texts_removed,  en_train_labels_removed = remove_unmatched_label(en_train_texts, en_train_labels,  en_remove_list) 

In [148]:
en_test_texts_removed,  en_test_labels_removed = remove_unmatched_label(en_test_texts, en_test_labels,  en_remove_list) 

In [16]:
sp_train_texts_removed,  sp_train_labels_removed = remove_unmatched_label(sp_train_texts, sp_train_labels, sp_remove_list) 

In [17]:
sp_test_texts_removed,  sp_test_labels_removed = remove_unmatched_label(sp_test_texts, sp_test_labels, sp_remove_list) 

# Rematch indices of Spanish tweets to corresponding indices of English tweets

In [18]:
def rematched_sp_labels(sp_labels):
    sp_labels_removed_rematched = []
    sp_labels_removed_rematched = [ 8 if  i == 11 else i for i in sp_labels]
    sp_labels_removed_rematched = [ 19 if  i == 13 else i for i in sp_labels_removed_rematched]
    sp_labels_removed_rematched = [ 7 if  i == 15 else i for i in sp_labels_removed_rematched]
    sp_labels_removed_rematched = [ 16 if  i == 18 else i for i in sp_labels_removed_rematched]
    sp_labels_removed_rematched = [ 13 if  i == 12 else i for i in sp_labels_removed_rematched]
    sp_labels_removed_rematched = [ 11 if  i == 9 else i for i in sp_labels_removed_rematched]
    sp_labels_removed_rematched = [ 9 if  i == 5 else i for i in sp_labels_removed_rematched]
    return sp_labels_removed_rematched

In [19]:
sp_train_labels_removed_rematched = rematched_sp_labels(sp_train_labels_removed)
sp_test_labels_removed_rematched = rematched_sp_labels(sp_test_labels_removed)

# Data Cleaning for Untranslated Data

In [188]:
p.set_options(p.OPT.URL, p.OPT.SMILEY, p.OPT.MENTION)

In [189]:
translator = str.maketrans("", "", punctuation)

In [190]:
get_stem_en = EnglishStemmer()

In [193]:
en_train_texts_special_char = []
for i in range(len(en_train_texts_removed)):
    en_train_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', en_train_texts_removed[i])
    
sp_train_texts_special_char = []
for i in range(len(sp_train_texts_removed)):
    sp_train_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', sp_train_texts_removed[i])
    
en_test_texts_special_char = []
for i in range(len(en_test_texts_removed)):
    en_test_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', en_test_texts_removed[i])
    
sp_test_texts_special_char = []
for i in range(len(sp_test_texts_removed)):
    sp_test_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', sp_test_texts_removed[i])

In [194]:
all_special_char = np.unique(en_train_texts_special_char + sp_train_texts_special_char + en_test_texts_special_char + sp_test_texts_special_char) 

In [195]:
all_special_char  = all_special_char.tolist() 

In [196]:
all_special_char += ['...']

In [198]:
special_char_single_string = ''
for i in range(len(all_special_char)):
    special_char_single_string += all_special_char[i]
    
special_char_single_string = '[' + special_char_single_string + ']'

In [202]:
def clean_english(texts):
    result = []
    for text in texts:
        #remove URL links, Smiley, and @user
        text = p.clean(text)
        #remove special characters
        text = re.sub(special_char_single_string, '', text)
        #make everything lower case
        text = text.lower()
        #remove stopwords
        text = ' '.join([i for i in text.split() if i not in (stopwords.words('english'))])
        #remove punctuation
        text = text.translate(translator)
        #change every word to stem word
        text = [get_stem_en.stem(i) for i  in word_tokenize(text)]
        result.append(' '.join(text))
    
    return result

In [203]:
en_train_texts_removed_cleaned = clean_english(en_train_texts_removed)

In [204]:
en_test_texts_removed_cleaned = clean_english(en_test_texts_removed)

In [207]:
with codecs.open("en_train_texts_removed_cleaned.txt",'w',"utf-8") as out_fs:
    for each in en_train_texts_removed_cleaned:
        out_fs.write(each + "\n")

In [208]:
with codecs.open("en_test_texts_removed_cleaned.txt",'w',"utf-8") as out_fs:
    for each in en_test_texts_removed_cleaned:
        out_fs.write(each + "\n")

In [209]:
get_stem_sp = SpanishStemmer()

In [210]:
def clean_spanish(texts):
    result = []
    
    for text in texts:
        #remove URL links, Smiley, and @user
        text = p.clean(text)
        #remove special characters
        text = re.sub(special_char_single_string, '', text)
        #make everything lower case
        text = text.lower()
        #remove stopwords
        text = ' '.join([i for i in text.split() if i not in (stopwords.words('spanish'))])
        #remove punctuation
        text = text.translate(translator)
        #change every word to stem word
        text = [get_stem_sp.stem(i) for i  in word_tokenize(text)]
        result.append(' '.join(text))
    
    return result

In [211]:
sp_train_texts_removed_cleaned = clean_spanish(sp_train_texts_removed)

In [212]:
sp_test_texts_removed_cleaned = clean_spanish(sp_test_texts_removed)

In [214]:
with codecs.open("sp_train_texts_removed_cleaned.txt",'w',"utf-8") as out_fs:
    for each in sp_train_texts_removed_cleaned:
        out_fs.write(each + "\n")

In [215]:
with codecs.open("sp_test_texts_removed_cleaned.txt",'w',"utf-8") as out_fs:
    for each in sp_test_texts_removed_cleaned:
        out_fs.write(each + "\n")

# Base Line Model

In [217]:
with open("/Users/ziranmin/Desktop/Assignment5/en_train_texts_removed_cleaned.txt", 'r') as f:
    en_train_texts_removed_cleaned_read = [l.strip() for l in f]

In [218]:
with open("/Users/ziranmin/Desktop/Assignment5/en_test_texts_removed_cleaned.txt", 'r') as f:
    en_test_texts_removed_cleaned_read = [l.strip() for l in f]

In [219]:
tf = TfidfVectorizer()

In [220]:
en_train_tf = tf.fit_transform(en_train_texts_removed_cleaned_read)

In [221]:
en_test_tf = tf.transform(en_test_texts_removed_cleaned_read)

In [241]:
LR_en = LogisticRegression(C=4)
LR_en.fit(en_train_tf, en_train_labels_removed)
    
en_pred = LR_en.predict(en_test_tf)
    
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(en_test_labels_removed), fmt='%d')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt
print()

Macro F-Score (official): 22.19
-----
Micro F-Score: 36.884
Precision: 36.884
Recall: 36.884



# Spanis to English Translation and Cleaning

In [25]:
trans = str.maketrans("", "", punctuation)

In [19]:
sp_to_en_translator =  Translator(from_lang="spanish", to_lang="english")

In [20]:
def sp_to_en(texts):
    result = []

    for txt in texts:
         #remove URL links, Smiley, and @user
        txt = p.clean(txt)
        #remove special characters
        txt = re.sub(special_char_single_string, '', txt)
        #remove punctuation
        txt = txt.translate(trans)
        
        #Spanish to English translation
        txt = sp_to_en_translator.translate(txt)
        #make everything lower case
        txt = txt.lower()
        #remove stopwords
        txt = ' '.join([i for i in txt.split() if i not in (stopwords.words('english'))])
        #change every word to stem word
        txt = [get_stem_en.stem(i) for i  in word_tokenize(txt)]
        
        result.append(' '.join(txt))
            
    return result

In [21]:
en_train_translated = sp_to_en(sp_train_texts_removed)

In [23]:
en_test_translated = sp_to_en(sp_test_texts_removed)

In [28]:
new_en_train_text = en_train_texts_removed_cleaned_read + en_train_translated
new_en_test_text = en_test_texts_removed_cleaned_read + en_test_translated

In [29]:
new_en_train_labels = en_train_labels_removed + sp_train_labels_removed
new_en_test_labels = en_test_labels_removed + sp_test_labels_removed

# New Modeling¶

In [30]:
tf = TfidfVectorizer()
new_en_train_tf = tf.fit_transform(new_en_train_text)
new_en_test_tf = tf.transform(new_en_test_text)

In [31]:
LR_new = LogisticRegression(C=4)
LR_new.fit(new_en_train_tf, new_en_train_labels)
    
en_pred = LR_new.predict(new_en_test_labels)
    
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(new_en_test_labels), fmt='%d')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt
print()

Macro F-Score (official): 15.643
-----
Micro F-Score: 30.78
Precision: 30.78
Recall: 30.78
